# Report02 - Nathan Yee

This notebook contains report03 for computational baysian statistics fall 2016

MIT License: https://opensource.org/licenses/MIT

In [1]:
from __future__ import print_function, division

% matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import math
import numpy as np

from thinkbayes2 import Pmf, Cdf, Suite, Joint
import thinkplot

## The sock problem

Created by Yuzhong Huang

There are two drawers of socks. The first drawer has 40 white socks and 10 black socks; the second drawer has 20 white socks and 30 black socks. We randomly get 2 socks from a drawer, and it turns out to be a pair(same color) but we don't know the color of these socks. What is the chance that we picked the first drawer.

To make calculating our likelihood easier, we start by defining a multiply function. The function is written in a functional way primarily for fun.

In [2]:
from functools import reduce
import operator

def multiply(items):
    """
    multiply takes a list of numbers, multiplies all of them, and returns the result
    
    Args:
        items (list): The list of numbers
        
    Return:
        the items multiplied together
    """
    return reduce(operator.mul, items, 1)

Next we define a drawer suite. This suite will allow us to take n socks up to the least number of socks in a drawer. To make our likelihood function simpler, we ignore the case where we take 11 black socks and that only drawer 2 is possible.

In [5]:
class Drawers(Suite):
    def Likelihood(self, data, hypo):
        """
        Likelihood returns the likelihood given a bayesian update 
        consisting of a particular hypothesis and new data. In the
        case of our drawer problem, the probabilities change with the
        number of pairs we take (without replacement) so we we start
        by defining lists for each color sock in each drawer.
        
        Args:
            data (int): The number of socks we take
            hypo (str): The hypothesis we are updating
            
        Return:
            the likelihood for a hypothesis
        """
        
        drawer1W = []
        drawer1B = []
        drawer2W = []
        drawer2B = []
        for i in range(data):
            drawer1W.append(40-i)
            drawer1B.append(10-i)
            drawer2W.append(20-i)
            drawer2B.append(30-i)
        
        if hypo == 'drawer1':
            return multiply(drawer1W)+multiply(drawer1B)
        if hypo == 'drawer2':
            return multiply(drawer2W)+multiply(drawer2B)


Next, define our hypotheses and create the drawer Suite.

In [6]:
hypos = ['drawer1','drawer2']
drawers = Drawers(hypos)
drawers.Print()

drawer1 0.5
drawer2 0.5


Next, update the drawers by taking two matching socks.

In [7]:
drawers.Update(2)
drawers.Print()

drawer1 0.5689655172413792
drawer2 0.43103448275862066


It seems that the drawer with many of a single sock (40 white 10 black) is more likely after the update. To confirm this suspicion, let's update with 3 more socks.

In [9]:
drawers.Update(3)
drawers.Print()

drawer1 0.7173913043478262
drawer2 0.28260869565217395


Now we can conclude that bayesian updates make the drawer with extreme amounts of one sock more likely when our information is just matching socks.

## Chess-playing twins

Allen Downey

Two identical twins are members of my chess club, but they never show up on the same day; in fact, they strictly alternate the days they show up.  I can't tell them apart except that one is a better player than the other:  Avery beats me 60% of the time and I beat Blake 70% of the time.  If I play one twin on Monday and win, and the other twin on Tuesday and lose, which twin did I play on which day?

To solve this problem, we first need to create our hypothesis. In this case, we have:  

hypo1: Avery Monday, Blake Tuesday  
hypo2: Blake Monday, Avery Tuesday  

We will abreviate Avery to A and Blake to B.

In [13]:
twins = Pmf()
twins['AB'] = 1
twins['BA'] = 1
twins.Normalize()
twins.Print()

AB 0.5
BA 0.5


Now we update our hypotheses with us winning the first day. We have a 40% chance of winning against Avery and a 70% chance of winning against Blake.

In [16]:
#win day 1
twins['AB'] *= .4
twins['BA'] *= .7
twins.Normalize()
twins.Print()

AB 0.3950617283950617
BA 0.6049382716049382


At this point in time, there is only a 36% chance that we play Avery the first day while a 64% chance that we played Blake the first day.

However, let's see what happens when we update with a loss.

In [15]:
#lose day 2
twins['AB'] *= .6
twins['BA'] *= .3
twins.Normalize()
twins.Print()

AB 0.5333333333333333
BA 0.4666666666666667


Interesting. Now there is a 53% chance that we played Avery then Blake and a 47% chance that we played Blake then Avery.

## Who was that person who saw a movie?

Nathan Yee

Every year the MPAA (Motion Picture Association of America) publishes a report about theatrical market statistics. Included in the report, both the gender and the ethnicity share of the top 5 most grossing films. If a randomly selected person went to Inside Out, what is the probability that they are both female and Asian?

Data:

| Gender                      | Male (%) | Female (%)  |
| :-------------------------- | :------- | :---------- |
| Furious 7                   | 56       | 44          |
| Inside Out                  | 46       | 54          |
| Avengers: Age of Ultron     | 58       | 42          |
| Star Wars: The Force Awakens| 58       | 42          |
| Jurassic World              | 55       | 45          |


| Ethnicity                   | Caucasian (%) | African-American (%) | Hispanic (%) | Asian (%) | Other (%) |
| :-------------------------- | :------------ | :------------------- | :----------- | :-------- | :-------- |
| Furious 7                   | 40            | 22                   | 25           | 8         | 5         |
| Inside Out                  | 54            | 15                   | 16           | 9         | 5         |
| Avengers: Age of Ultron     | 50            | 16                   | 20           | 10        | 5         |
| Star Wars: The Force Awakens| 61            | 12                   | 15           | 7         | 5         |
| Jurassic World              | 39            | 16                   | 19           | 11        | 6         |


| Demographic                 | Caucasian (%) | African-American (%) | Hispanic (%) | Asian (%) | Other (%) |
| :-------------------------- | :------------ | :------------------- | :----------- | :-------- | :-------- |
| Population United States    | 63.7          | 12.2                 | 16.3         | 4.7       | 3.1       |

Note:
Demographic data was hastily gathered from the internet. There may be errors within 2% due to rounding. Also note that certian races were combined to fit our previous data.

To make writing code easier, we will encoude data in a numerical structure. The first item in the tuple corresponds to gender, the second item in the tuple corresponds to ethnicity.

| Gender                      | Male | Female |
| :-------------------------- | :--- | :----- |
| Encoding number             | 0    | 1      |

| Ethnicity                   | Caucasian | African-American | Hispanic | Asian | Other |
| :-------------------------- | :-------- | :--------------- | :------- | :---- | :---- |
| Encoding number             | 0         | 1                | 2        | 3     | 4     |


Such that a (female, asian) = (1, 3)

In [92]:
class Movie(Suite, Joint):
    def Likelihood(self, data, hypo):
        """
        Likelihood returns the likelihood given a bayesian update consisting of a particular
        hypothesis and data. In this case, we need to calculate the probability of seeing a
        gender seeing a movie. Then we calculat the probability that an ethnicity saw a
        movie. Finally we multiply the two to calculate the a person of a gender and 
        ethnicity saw a movie.
        
        Args:
            data (str): The title of the movie
            hypo (str): The hypothesis we are updating
            
        Return:
            the likelihood for a hypothesis
        """
        
        movie = data        
        gender = hypo[0]
        ethnicity = hypo[1]
        
        # first calculate update based on gender
        movies_gender = {'Furious 7'                    : {0:56, 1:44},
                         'Inside Out'                   : {0:46, 1:54},
                         'Avengers: Age of Ultron'      : {0:58, 1:42},
                         'Star Wars: The Force Awakens' : {0:58, 1:42},
                         'Jurassic World'               : {0:55, 1:45}
                         }
        
        like_gender = movies_gender[movie][gender]
        
        # second calculate update based on ethnicity
        movies_ethnicity = {'Furious 7'                    : {0:40, 1:22, 2:25, 3:8 , 4:5},
                            'Inside Out'                   : {0:54, 1:15, 2:16, 3:9 , 4:4},
                            'Avengers: Age of Ultron'      : {0:50, 1:16, 2:20, 3:10, 4:5},
                            'Star Wars: The Force Awakens' : {0:61, 1:12, 2:15, 3:7 , 4:5},
                            'Jurassic World'               : {0:39, 1:16, 2:19, 3:11, 4:6}
                 }
        
        like_ethnicity = movies_ethnicity[movie][ethnicity]
        
        # multiply the two together and return
        return like_gender * like_ethnicity
        
    

Next we make our hypotheses and input them as tuples into the Movie class.

In [105]:
genders = range(0,2)
ethnicities = range(0,5)
pairs = [(gender, ethnicity) for gender in genders for ethnicity in ethnicities]

movie = Movie(pairs)
movie.Print()

(0, 0) 0.1
(0, 1) 0.1
(0, 2) 0.1
(0, 3) 0.1
(0, 4) 0.1
(1, 0) 0.1
(1, 1) 0.1
(1, 2) 0.1
(1, 3) 0.1
(1, 4) 0.1


We decided that we are picking a random person in the United states. So, we can use population demographics of the United States as an informed prior. We will assume that the United States is 50% male and 50% female. Population percent is defined in the order which we enumerate ethnicities.

In [106]:
# If there is time, I will update this code to make it look nicer. For now, it works.
population_percent = [63.7, 12.2, 16.3, 4.7, 3.1, 63.7, 12.2, 16.3, 4.7, 3.1]

for i in range(len(population_percent)):
    movie[pairs[i]] = population_percent[i]

movie.Normalize()
movie.Print()

(0, 0) 0.3185
(0, 1) 0.061
(0, 2) 0.0815
(0, 3) 0.0235
(0, 4) 0.015500000000000002
(1, 0) 0.3185
(1, 1) 0.061
(1, 2) 0.0815
(1, 3) 0.0235
(1, 4) 0.015500000000000002


Next update with the two movies

In [107]:
movie.Update('Inside Out')

1969.1499999999999

In [108]:
movie.Normalize()
movie.Print()

(0, 0) 0.4017743696518802
(0, 1) 0.021374704821877456
(0, 2) 0.03046187441281771
(0, 3) 0.004940710458827413
(0, 4) 0.0014483406545971614
(1, 0) 0.4716481730695985
(1, 1) 0.02509204479089962
(1, 2) 0.0357595917020034
(1, 3) 0.005799964451666963
(1, 4) 0.0017002259858314502


Given that a random person has seen Inside Out, the probability that the person is both female and Asian is .58%. Interestingly, when we update our hypotheses with our data, the the chance that the randomly selected person is caucasian goes up to 87%. It seems that our model just increases the chance that the randomly selected person is caucasian after seeing a movie.